# Import necessary packages

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from random import sample

import re

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import RandomizedSearchCV 

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [ ]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/calendar.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [ ]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOBBIES')]

In [ ]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Hobbies/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [ ]:
sales_smooth

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
54,HOBBIES_1_055_CA_1_validation,HOBBIES_1_055,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,2,...,2,2,2,4,6,1,1,1,8,3
62,HOBBIES_1_064_CA_1_validation,HOBBIES_1_064,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,3,4,2,0,5,0,0,0,0
71,HOBBIES_1_074_CA_1_validation,HOBBIES_1_074,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,1,3,1,1,0,3
129,HOBBIES_1_134_CA_1_validation,HOBBIES_1_134,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,8,16,0,1,2,2,2,3
136,HOBBIES_1_141_CA_1_validation,HOBBIES_1_141,HOBBIES_1,HOBBIES,CA_1,CA,2,1,3,0,...,1,2,5,1,3,1,0,1,2,1
142,HOBBIES_1_147_CA_1_validation,HOBBIES_1_147,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,6,8,7,13,7,3,11,7,6,9
143,HOBBIES_1_148_CA_1_validation,HOBBIES_1_148,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,2,0,0,1,1,1,2,1,2
150,HOBBIES_1_155_CA_1_validation,HOBBIES_1_155,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,4,3,3,4,1,2,3,3,5
153,HOBBIES_1_158_CA_1_validation,HOBBIES_1_158,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,4,5,6,5,3,5,3,3,1,8
188,HOBBIES_1_194_CA_1_validation,HOBBIES_1_194,HOBBIES_1,HOBBIES,CA_1,CA,5,5,2,5,...,2,1,2,2,1,0,2,0,1,2


# User-defined functions to calculate Metrics

In [ ]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

# Format sales data and then merge with calendar data

In [ ]:
def data_cleaning(sales_pattern):

    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()

    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')

    df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
    df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
    df_pattern.reset_index(inplace=True)

    calendar['Date'] = pd.to_datetime(calendar['date'])

    dataframe = pd.merge(df_pattern,
                         calendar[['wday', 'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA', 'Date']],
                         on = 'Date')

    dataframe['weekend'] = np.where(((dataframe['wday'] == 1) | (dataframe['wday'] == 2)), 1, 0)
    dataframe.fillna('No event', inplace=True)
    dataframe['year'] = dataframe['year'].apply(str)

    dataframe['quarter'] = dataframe['Date'].dt.quarter
    dataframe['quarter'] = dataframe['quarter'].apply(str)

    dataframe = pd.get_dummies(dataframe)
    
    return dataframe

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---



In [ ]:
data_pattern = data_cleaning(sales_smooth)

In [ ]:
data_pattern

,Date,FOODS_1_004_CA_1,FOODS_1_004_CA_3,FOODS_1_004_CA_4,FOODS_1_012_CA_1,FOODS_1_012_CA_2,FOODS_1_012_CA_3,FOODS_1_012_CA_4,FOODS_1_018_CA_1,FOODS_1_018_CA_4,...,event_name_1_VeteransDay,event_type_1_Cultural,event_type_1_National,event_type_1_No event,event_type_1_Religious,event_type_1_Sporting,quarter_1,quarter_2,quarter_3,quarter_4
0,2011-01-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,2.0,...,0,0,0,1,0,0,1,0,0,0
1,2011-01-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0,0,0,1,0,0,1,0,0,0
2,2011-01-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,...,0,0,0,1,0,0,1,0,0,0
3,2011-02-01,4.0,0.0,1.0,4.0,21.0,11.0,3.0,8.0,8.0,...,0,0,0,1,0,0,1,0,0,0
4,2011-02-02,9.0,0.0,3.0,7.0,24.0,18.0,8.0,14.0,6.0,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,2016-04-20,10.0,0.0,1.0,7.0,8.0,7.0,10.0,13.0,8.0,...,0,0,0,1,0,0,0,1,0,0
1909,2016-04-21,3.0,0.0,1.0,2.0,6.0,11.0,1.0,15.0,7.0,...,0,0,0,1,0,0,0,1,0,0
1910,2016-04-22,7.0,0.0,2.0,5.0,13.0,10.0,3.0,14.0,9.0,...,0,0,0,1,0,0,0,1,0,0
1911,2016-04-23,6.0,0.0,3.0,4.0,9.0,7.0,6.0,15.0,7.0,...,0,0,0,1,0,0,0,1,0,0


# Random Forest Hyperparameters to find optimum parameters

In [ ]:
not_id_cols = [
 'Date',
 'wday',
 'month',
 'snap_CA',
 'weekend',
 'year_2011',
 'year_2012',
 'year_2013',
 'year_2014',
 'year_2015',
 'year_2016',
 'event_name_1_Chanukah End',
 'event_name_1_Christmas',
 'event_name_1_Cinco De Mayo',
 'event_name_1_ColumbusDay',
 'event_name_1_Easter',
 'event_name_1_Eid al-Fitr',
 'event_name_1_EidAlAdha',
 "event_name_1_Father's day",
 'event_name_1_Halloween',
 'event_name_1_IndependenceDay',
 'event_name_1_LaborDay',
 'event_name_1_LentStart',
 'event_name_1_LentWeek2',
 'event_name_1_MartinLutherKingDay',
 'event_name_1_MemorialDay',
 "event_name_1_Mother's day",
 'event_name_1_NBAFinalsEnd',
 'event_name_1_NBAFinalsStart',
 'event_name_1_NewYear',
 'event_name_1_No event',
 'event_name_1_OrthodoxChristmas',
 'event_name_1_OrthodoxEaster',
 'event_name_1_Pesach End',
 'event_name_1_PresidentsDay',
 'event_name_1_Purim End',
 'event_name_1_Ramadan starts',
 'event_name_1_StPatricksDay',
 'event_name_1_SuperBowl',
 'event_name_1_Thanksgiving',
 'event_name_1_ValentinesDay',
 'event_name_1_VeteransDay',
 'event_type_1_Cultural',
 'event_type_1_National',
 'event_type_1_No event',
 'event_type_1_Religious',
 'event_type_1_Sporting',
 'quarter_1',
 'quarter_2',
 'quarter_3',
 'quarter_4'
]

In [ ]:
end_train_date = '2016-04-10'

In [ ]:
lags = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28]
moving_averages = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
stds = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
maximums = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
minimums = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
totals = [2, 3, 4, 5, 6, 7, 14, 21, 28]
num_days_zeros = [2, 3, 4, 5, 6, 7, 14, 21, 28] 

In [ ]:
correlation_threshold = 0.8

In [ ]:
n_estimators = [50, 100, 200, 300]
max_depth = list(range(4,15)) + [None]
min_samples_leaf = range(2,12)
max_features = range(10,30)
bootstrap = [True] #We force bootstrap
max_samples = [0.6, 0.7, 0.8, 0.9, 1]

In [ ]:
# User-defined funtion to find optimum parameters on training data
def Random_Forest_Pipeline(list_pattern, not_id_cols, lags, moving_averages, stds,
                          maximums, minimums, totals, num_days_zeros, correlation_threshold,
                          end_train_date, n_estimators, max_depth, min_samples_leaf,
                          max_features, bootstrap, max_samples):
    
    list_pattern_temp = list_pattern.copy()
    list_pattern_temp = [(re.sub(r'_validation', r"", x)) for x in list_pattern_temp]
    
    df_output_hyper_params = pd.DataFrame()
    
    for product in list_pattern_temp:
        print('Currently Running Product: %s' % product)
        print('Progressing: {0} %'.format(round(list_pattern_temp.index(product) / len(list_pattern_temp) * 100, 2)))
        dataframe_product = pd.concat([data_pattern[[product]], data_pattern[not_id_cols]], axis=1, ignore_index=False)

        # Create Lag variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (t-{lag})': dataframe_product[product].shift(lag)
            for lag in lags
        })

        # Create Rolling Moving Average variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
            for moving_average in moving_averages
        })

        # Create Rolling Standard Deviation variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
            for std in stds
        })

        # Create Rolling Maximum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
            for maximum in maximums
        })

        # Create Rolling Minimum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
            for minimum in minimums
        })

        # Create Rolling Total variables 
        dataframe_product = dataframe_product.assign(**{
            f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
            for total in totals
        })

        # Create Zero Count variables
        count_func = lambda x: (x==0).sum()

        dataframe_product = dataframe_product.assign(**{
            f'{product} (zero_count_in_{num_days_zero})': dataframe_product[product].shift(1).rolling(num_days_zero).apply(count_func)
            for num_days_zero in num_days_zeros
        })
        
        dataframe_product.dropna(inplace=True)
        dataframe_product.set_index('Date', inplace=True)

        train_data = dataframe_product[dataframe_product.index <= end_train_date]

        corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
        high_corr_var=np.where(corr_matrix > correlation_threshold)
        high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
        
        train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
        
        X_train = train_data.drop(product, axis=1)
        y_train = train_data[product]

        param_dist = {'n_estimators': n_estimators,
                      'max_depth': max_depth,
                      'min_samples_leaf': min_samples_leaf,
                      'max_features': max_features,
                      'bootstrap': bootstrap,
                      'max_samples': max_samples}

        forest = RandomForestRegressor(n_jobs=1)

        forest_cv = RandomizedSearchCV(forest, param_dist, cv=5, n_jobs=-1, verbose=2, n_iter=300, scoring='neg_mean_absolute_error')
        forest_cv.fit(X_train,y_train)
            
        best_params_temp = forest_cv.best_params_
        
        df_output_temp = pd.DataFrame.from_dict(best_params_temp, orient='index').T
        df_output_temp['Product'] = product
        
        df_output_hyper_params = df_output_hyper_params.append(df_output_temp, ignore_index=True)
        
    return df_output_hyper_params

**Sample 30 products due to limited computational power**

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---



In [ ]:
list_pattern_sample = sample(list_smooth, 30)

In [ ]:
list_pattern_sample

['HOBBIES_1_337_CA_3_validation',
 'HOBBIES_1_055_CA_3_validation',
 'HOBBIES_1_354_CA_4_validation',
 'HOBBIES_1_354_CA_2_validation',
 'HOBBIES_1_236_CA_4_validation',
 'HOBBIES_1_194_CA_3_validation',
 'HOBBIES_1_040_CA_3_validation',
 'HOBBIES_1_134_CA_1_validation',
 'HOBBIES_1_236_CA_3_validation',
 'HOBBIES_1_004_CA_3_validation',
 'HOBBIES_1_155_CA_1_validation',
 'HOBBIES_1_064_CA_1_validation',
 'HOBBIES_1_194_CA_1_validation',
 'HOBBIES_1_148_CA_3_validation',
 'HOBBIES_1_074_CA_1_validation',
 'HOBBIES_1_155_CA_3_validation',
 'HOBBIES_1_147_CA_3_validation',
 'HOBBIES_1_337_CA_4_validation',
 'HOBBIES_1_147_CA_4_validation',
 'HOBBIES_1_005_CA_4_validation',
 'HOBBIES_1_064_CA_3_validation',
 'HOBBIES_1_055_CA_1_validation',
 'HOBBIES_1_158_CA_1_validation',
 'HOBBIES_1_201_CA_3_validation',
 'HOBBIES_1_142_CA_3_validation',
 'HOBBIES_1_048_CA_4_validation',
 'HOBBIES_1_164_CA_3_validation',
 'HOBBIES_1_337_CA_1_validation',
 'HOBBIES_1_236_CA_1_validation',
 'HOBBIES_1_35

**Start Hyperparameters tuning process**

In [18]:
start = timer()

df_hyper_params = Random_Forest_Pipeline(list_pattern_sample, not_id_cols, lags, moving_averages, stds,
                                         maximums, minimums, totals, num_days_zeros, correlation_threshold,
                                         end_train_date, n_estimators, max_depth, min_samples_leaf,
                                         max_features, bootstrap, max_samples)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: HOBBIES_1_337_CA_3
Progressing: 0.0 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running Product: HOBBIES_1_055_CA_3
Progressing: 3.33 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running Product: HOBBIES_1_354_CA_4
Progressing: 6.67 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running Product: HOBBIES_1_354_CA_2
Progressing: 10.0 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running Product: HOBBIES_1_236_CA_4
Progressing: 13.33 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running Product: HOBBIES_1_194_CA_3
Progressing: 16.67 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running Product: HOBBIES_1_040_CA_3
Progressing: 20.0 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running Product: HOBBIES_1_134_CA_1
Progressing: 23.33 %
Fitting 5 folds for ea

In [19]:
df_hyper_params

,n_estimators,min_samples_leaf,max_samples,max_features,max_depth,bootstrap,Product
0,300,11,0.6,21,6,True,HOBBIES_1_337_CA_3
1,50,10,0.7,14,7,True,HOBBIES_1_055_CA_3
2,300,11,0.9,29,6,True,HOBBIES_1_354_CA_4
3,100,7,0.7,25,8,True,HOBBIES_1_354_CA_2
4,50,9,0.9,15,10,True,HOBBIES_1_236_CA_4
5,50,8,0.6,15,7,True,HOBBIES_1_194_CA_3
6,200,4,0.9,28,12,True,HOBBIES_1_040_CA_3
7,100,5,1,27,12,True,HOBBIES_1_134_CA_1
8,50,6,0.6,28,7,True,HOBBIES_1_236_CA_3
9,50,4,0.8,14,5,True,HOBBIES_1_004_CA_3


In [20]:
df_hyper_params.to_csv('Random_Forest_Hyperparams_Smooth.csv')

# Decide optimum set of parameters for Random Forest algorithm

In [21]:
# See if any values larger than 1 in column "Product"
# If yes, the corresponding set of parameters occur more than once and those sets should be considered first
df_hyper_params.groupby(['n_estimators', 'min_samples_leaf', 'max_samples', 
                         'max_features', 'max_depth', 'bootstrap'])[['Product']].count().sort_values(by = 'Product', ascending=False)

Product
n_estimators min_samples_leaf max_samples max_features max_depth bootstrap         
50           3                0.6         23           4         True             1
100          4                0.6         26           5         True             1
300          11               0.6         21           6         True             1
             7                0.9         25           6         True             1
             6                0.9         14           7         True             1
200          9                1.0         12           5         True             1
             8                0.7         19           10        True             1
             6                1.0         22           5         True             1
             5                0.7         27           5         True             1
             4                0.9         28           12        True             1
100          10               0.6         21           9         True             1
             7                0.7         25           8         True             1
             5                1.0         27           12        True             1
             4                0.7         22           5         True             1
             2                0.9         25           7         True             1
50           4                0.8         14           5         True             1
100          2                0.8         22           4         True             1
50           11               1.0         29           13        True             1
                              0.9         27           5         True             1
                              0.8         26           7         True             1
                              0.6         23           14        True             1
             10               0.7         14           7         True             1
             9                0.9         15           10        True             1
                              0.6         12           13        True             1
             8                0.6         15           7         True             1
             7                0.6         25           7         True             1
             6                0.6         28           7         True             1
                                          24           14        True             1
300          11               0.9         29           6         True             1

In [22]:
# See what value of parameter "n_estimators" occurs the most frequently
df_hyper_params.groupby(['n_estimators'])[['Product']].count()

,Product
n_estimators,
50,13
100,8
200,5
300,4


In [23]:
# See what value of parameter "min_samples_leaf" occurs the most frequently
df_hyper_params.groupby(['min_samples_leaf'])[['Product']].count()

,Product
min_samples_leaf,
2,2
3,1
4,4
5,2
6,4
7,3
8,2
9,3
10,2


In [24]:
# See what value of parameter "max_samples" occurs the most frequently
df_hyper_params.groupby(['max_samples'])[['Product']].count()

,Product
max_samples,
0.6,10
0.7,5
0.8,3
0.9,8
1.0,4


In [25]:
# See what value of parameter "max_features" occurs the most frequently
df_hyper_params.groupby(['max_features'])[['Product']].count()

,Product
max_features,
12,2
14,3
15,3
19,1
21,2
22,3
23,2
24,1
25,4


In [26]:
# See what value of parameter "max_depth" occurs the most frequently
df_hyper_params.groupby(['max_depth'])[['Product']].count()

,Product
max_depth,
4,2
5,7
6,3
7,7
8,1
9,1
10,2
12,2
13,2


# Implement Random Forest on Test Set with optimum set of parameters

**Specify optimum set of parameters**

In [27]:
n_estimators_chosen = 50
min_samples_leaf_chosen = 5
max_samples_choose = 0.6
max_features_choose = 21
max_depth_choose = 6
bootstrap = [True]

In [28]:
# User-defined function to run Random Forest on Test Data

def Random_Forest_Test_Data(list_pattern, not_id_cols, lags, moving_averages, stds,
                          maximums, minimums, totals, num_days_zeros, correlation_threshold,
                          end_train_date, n_estimators_chosen, min_samples_leaf_chosen,
                          max_samples_choose, max_features_choose, max_depth_choose, bootstrap):
    
    list_pattern_temp = list_pattern.copy()
    list_pattern_temp = [(re.sub(r'_validation', r"", x)) for x in list_pattern_temp]
    
    df_output_rf_pattern = pd.DataFrame()
    
    for product in list_pattern_temp:
        print('Currently Running Product: %s' % product)
        print('Progressing: {0} %'.format(round(list_pattern_temp.index(product) / len(list_pattern_temp) * 100, 2)))

        dataframe_product = pd.concat([data_pattern[[product]], data_pattern[not_id_cols]], axis=1, ignore_index=False)

        # Create Lag variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (t-{lag})': dataframe_product[product].shift(lag)
            for lag in lags
        })

        # Create Rolling Moving Average variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
            for moving_average in moving_averages
        })

        # Create Rolling Standard Deviation variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
            for std in stds
        })

        # Create Rolling Maximum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
            for maximum in maximums
        })

        # Create Rolling Minimum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
            for minimum in minimums
        })

        # Create Rolling Total variables 
        dataframe_product = dataframe_product.assign(**{
            f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
            for total in totals
        })

        # Create Zero Count variables
        count_func = lambda x: (x==0).sum()

        dataframe_product = dataframe_product.assign(**{
            f'{product} (zero_count_in_{num_days_zero})': dataframe_product[product].shift(1).rolling(num_days_zero).apply(count_func)
            for num_days_zero in num_days_zeros
        })
        
        dataframe_product.dropna(inplace=True)
        dataframe_product.set_index('Date', inplace=True)

        train_data = dataframe_product[dataframe_product.index <= end_train_date]
        test_data = dataframe_product[dataframe_product.index > end_train_date]

        corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
        high_corr_var=np.where(corr_matrix > correlation_threshold)
        high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
        
        train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
        test_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
        
        X_train = train_data.drop(product, axis=1)
        y_train = train_data[product]

        X_test = test_data.drop(product, axis=1)
        y_test = test_data[product]

        forest = RandomForestRegressor(n_jobs=1,
                                      n_estimators = n_estimators_chosen,
                                      min_samples_leaf = min_samples_leaf_chosen,
                                      max_samples = max_samples_choose,
                                      max_features = max_features_choose,
                                      max_depth = max_depth_choose,
                                      bootstrap = bootstrap,
                                      random_state = 0)

        forest.fit(X_train,y_train)
        y_pred = forest.predict(X_test)
        df_output_temp = pd.DataFrame({'Actual Data': y_test, 
                                       'Forecast': y_pred, 
                                       'Product': [product for num_rows in range(len(y_pred))]
                                       })
        
        df_output_rf_pattern = df_output_rf_pattern.append(df_output_temp, ignore_index=False)
        
    return df_output_rf_pattern

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---



In [29]:
# Start running Random Forest on Test Set

start = timer()

df_forecast_values = Random_Forest_Test_Data(list_smooth, not_id_cols, lags, moving_averages, stds,
                                             maximums, minimums, totals, num_days_zeros, correlation_threshold,
                                             end_train_date, n_estimators_chosen, min_samples_leaf_chosen,
                                             max_samples_choose, max_features_choose, max_depth_choose, bootstrap)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: HOBBIES_1_004_CA_2
Progressing: 0.0 %
Currently Running Product: HOBBIES_1_004_CA_3
Progressing: 2.44 %
Currently Running Product: HOBBIES_1_005_CA_4
Progressing: 4.88 %
Currently Running Product: HOBBIES_1_040_CA_3
Progressing: 7.32 %
Currently Running Product: HOBBIES_1_048_CA_4
Progressing: 9.76 %
Currently Running Product: HOBBIES_1_055_CA_1
Progressing: 12.2 %
Currently Running Product: HOBBIES_1_055_CA_3
Progressing: 14.63 %
Currently Running Product: HOBBIES_1_056_CA_3
Progressing: 17.07 %
Currently Running Product: HOBBIES_1_064_CA_1
Progressing: 19.51 %
Currently Running Product: HOBBIES_1_064_CA_3
Progressing: 21.95 %
Currently Running Product: HOBBIES_1_074_CA_1
Progressing: 24.39 %
Currently Running Product: HOBBIES_1_134_CA_1
Progressing: 26.83 %
Currently Running Product: HOBBIES_1_134_CA_3
Progressing: 29.27 %
Currently Running Product: HOBBIES_1_134_CA_4
Progressing: 31.71 %
Currently Running Product: HOBBIES_1_141_CA_1
Progressing: 34.15 %
Cu

In [30]:
df_forecast_values

,Actual Data,Forecast,Product
Date,,,
2016-04-11,1.0,4.482305,HOBBIES_1_004_CA_2
2016-04-12,3.0,3.757497,HOBBIES_1_004_CA_2
2016-04-13,12.0,3.963619,HOBBIES_1_004_CA_2
2016-04-14,0.0,4.048859,HOBBIES_1_004_CA_2
2016-04-15,3.0,4.063936,HOBBIES_1_004_CA_2
...,...,...,...
2016-04-20,0.0,0.003478,HOBBIES_1_354_CA_4
2016-04-21,0.0,0.004902,HOBBIES_1_354_CA_4
2016-04-22,0.0,0.005435,HOBBIES_1_354_CA_4


In [31]:
df_forecast_values.to_csv('Random Forest_Smooth_Forecast_Values.csv')

In [32]:
# User-defined funcion to calculate metrics

def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [33]:
start = timer()

df_forecast_metrics = df_forecast_values.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.0018325933333168602 minutes


In [34]:
df_forecast_metrics

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_004_CA_2,0.584376,66.0279,69.5799,90.2014,2.876929,3.702083
1,HOBBIES_1_004_CA_3,0.633777,33.7342,39.5979,25.6014,2.096339,2.805758
2,HOBBIES_1_005_CA_4,inf,inf,200.0000,NaN,0.039247,0.039281
3,HOBBIES_1_040_CA_3,1.006534,90.3299,101.2315,43.9806,1.161385,1.329679
4,HOBBIES_1_048_CA_4,0.757222,108.3045,99.3369,143.9850,4.951065,6.601857
5,HOBBIES_1_055_CA_1,0.582075,80.8839,97.5009,52.8615,1.791001,2.500705
6,HOBBIES_1_055_CA_3,0.862626,67.5623,86.2908,52.4367,1.592540,1.994755
7,HOBBIES_1_056_CA_3,0.780223,64.8185,81.4875,56.0811,1.620463,1.833454
8,HOBBIES_1_064_CA_1,0.644147,65.9012,79.4729,45.6796,0.941446,1.054062
9,HOBBIES_1_064_CA_3,0.860064,62.4194,80.0763,35.4992,2.050922,2.898015


In [35]:
df_forecast_values.to_csv('Random Forest_Smooth_Forecast_Metrics.csv')